In [1]:
 import os
from PIL import Image
from tqdm import tqdm
import warnings

# =========================
# CONFIGURATION
# =========================
PROCESSED_DIR = r"D:\Final_Semester_Project\AI_Attendance_System\ai-ml-model\DataSets\SecondLap\First300"
TARGET_SIZE = (112, 112)  # Width, Height_
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp"}
RESIZE_METHOD = Image.Resampling.LANCZOS  # High-quality resampling

# Suppress warnings
warnings.filterwarnings('ignore')

# =========================
# RESIZE FUNCTIONS
# =========================
def is_image_file(filename: str) -> bool:
    return os.path.splitext(filename.lower())[1] in IMAGE_EXTENSIONS

def resize_image_preserve_aspect(input_path: str, output_path: str, target_size: tuple):
    """
    Resize image to target size while preserving aspect ratio.
    Adds padding if necessary to maintain exact dimensions.
    """
    try:
        with Image.open(input_path) as img:
            # Convert RGBA to RGB if necessary
            if img.mode in ('RGBA', 'LA'):
                # Create a white background
                background = Image.new('RGB', img.size, (255, 255, 255))
                # Paste the image
                background.paste(img, mask=img.split()[-1])
                img = background
            elif img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Get original dimensions
            original_width, original_height = img.size
            
            # Calculate scaling factor
            width_ratio = target_size[0] / original_width
            height_ratio = target_size[1] / original_height
            
            # Use the smaller ratio to fit within target size
            ratio = min(width_ratio, height_ratio)
            
            new_width = int(original_width * ratio)
            new_height = int(original_height * ratio)
            
            # Resize the image
            img_resized = img.resize((new_width, new_height), RESIZE_METHOD)
            
            # Create new image with white background
            new_img = Image.new('RGB', target_size, (255, 255, 255))
            
            # Calculate position to center the image
            left = (target_size[0] - new_width) // 2
            top = (target_size[1] - new_height) // 2
            
            # Paste the resized image onto center of white background
            new_img.paste(img_resized, (left, top))
            
            # Save with optimization
            new_img.save(output_path, optimize=True, quality=95)
            return True
            
    except Exception:
        return False

def resize_image_stretch(input_path: str, output_path: str, target_size: tuple):
    """
    Simple stretch resize (changes aspect ratio)
    """
    try:
        with Image.open(input_path) as img:
            # Convert to RGB if necessary
            if img.mode in ('RGBA', 'LA'):
                background = Image.new('RGB', img.size, (255, 255, 255))
                background.paste(img, mask=img.split()[-1])
                img = background
            elif img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Direct resize (stretches/squishes)
            img_resized = img.resize(target_size, RESIZE_METHOD)
            img_resized.save(output_path, optimize=True, quality=95)
            return True
            
    except Exception:
        return False

def resize_image_crop(input_path: str, output_path: str, target_size: tuple):
    """
    Resize by cropping to maintain aspect ratio
    """
    try:
        with Image.open(input_path) as img:
            # Convert to RGB if necessary
            if img.mode in ('RGBA', 'LA'):
                background = Image.new('RGB', img.size, (255, 255, 255))
                background.paste(img, mask=img.split()[-1])
                img = background
            elif img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Calculate cropping
            original_width, original_height = img.size
            target_width, target_height = target_size
            
            # Calculate aspect ratios
            img_ratio = original_width / original_height
            target_ratio = target_width / target_height
            
            if img_ratio > target_ratio:
                # Image is wider than target
                new_height = original_height
                new_width = int(new_height * target_ratio)
            else:
                # Image is taller than target
                new_width = original_width
                new_height = int(new_width / target_ratio)
            
            # Calculate crop box
            left = (original_width - new_width) // 2
            top = (original_height - new_height) // 2
            right = left + new_width
            bottom = top + new_height
            
            # Crop and resize
            img_cropped = img.crop((left, top, right, bottom))
            img_resized = img_cropped.resize(target_size, RESIZE_METHOD)
            img_resized.save(output_path, optimize=True, quality=95)
            return True
            
    except Exception:
        return False

# =========================
# MAIN LOGIC
# =========================
def main():
    # Check if processed directory exists
    if not os.path.isdir(PROCESSED_DIR):
        print(f"Error: Directory not found: {PROCESSED_DIR}")
        return
    
    # Get all class directories
    class_dirs = sorted([
        d for d in os.listdir(PROCESSED_DIR)
        if os.path.isdir(os.path.join(PROCESSED_DIR, d))
    ])
    
    if not class_dirs:
        print(f"No class directories found in: {PROCESSED_DIR}")
        return
    
    # Ask user for resize method
    print("\n" + "="*60)
    print("IMAGE RESIZING TOOL")
    print("="*60)
    print(f"Target size: {TARGET_SIZE[0]}x{TARGET_SIZE[1]}")
    print(f"Source directory: {PROCESSED_DIR}")
    print(f"Total directories: {len(class_dirs)}")
    print("\nSelect resize method:")
    print("  1. Preserve aspect ratio (add padding)")
    print("  2. Stretch to fit (change aspect ratio)")
    print("  3. Crop to fit (maintain aspect ratio)")
    
    choice = input("\nEnter choice (1, 2, or 3): ").strip()
    
    if choice == '1':
        resize_func = resize_image_preserve_aspect
        method_name = "Preserve aspect with padding"
    elif choice == '2':
        resize_func = resize_image_stretch
        method_name = "Stretch to fit"
    elif choice == '3':
        resize_func = resize_image_crop
        method_name = "Crop to fit"
    else:
        print("Invalid choice. Using method 1 (preserve aspect).")
        resize_func = resize_image_preserve_aspect
        method_name = "Preserve aspect with padding"
    
    # Count total images first
    print("\nCounting images...")
    total_images = 0
    images_to_process = []
    
    for class_name in class_dirs:
        class_path = os.path.join(PROCESSED_DIR, class_name)
        image_files = [f for f in os.listdir(class_path) if is_image_file(f)]
        total_images += len(image_files)
        images_to_process.extend([(class_name, f) for f in image_files])
    
    if total_images == 0:
        print("No images found to process.")
        return
    
    # Show summary
    print(f"\nSummary:")
    print(f"  Directories: {len(class_dirs)}")
    print(f"  Total images: {total_images}")
    print(f"  Target size: {TARGET_SIZE[0]}x{TARGET_SIZE[1]}")
    print(f"  Method: {method_name}")
    print(f"  Source: {PROCESSED_DIR}")
    
    # Confirm
    confirm = input(f"\nProceed with resizing {total_images} images? (y/n): ").strip().lower()
    if confirm not in ['y', 'yes']:
        print("Operation cancelled.")
        return
    
    # Process images with single progress bar
    print(f"\nProcessing {total_images} images...")
    
    success_count = 0
    fail_count = 0
    
    # Single overall progress bar
    with tqdm(total=total_images, desc="Overall Progress", unit="img") as pbar:
        for class_name, img_filename in images_to_process:
            input_path = os.path.join(PROCESSED_DIR, class_name, img_filename)
            
            # Process the image
            if resize_func(input_path, input_path, TARGET_SIZE):  # Overwrite original
                success_count += 1
            else:
                fail_count += 1
            
            # Update progress bar
            pbar.update(1)
    
    # Print summary
    print("\n" + "="*60)
    print("RESIZE COMPLETE")
    print("="*60)
    print(f"Successfully resized: {success_count} images")
    print(f"Failed: {fail_count} images")
    
    if fail_count == 0:
        print("✅ All images processed successfully!")
    else:
        print(f"⚠ {fail_count} images failed to process")
    
    print(f"\nAll images resized to {TARGET_SIZE[0]}x{TARGET_SIZE[1]} using: {method_name}")

if __name__ == "__main__":
    main()


IMAGE RESIZING TOOL
Target size: 112x112
Source directory: D:\Final_Semester_Project\AI_Attendance_System\ai-ml-model\DataSets\SecondLap\First300
Total directories: 540

Select resize method:
  1. Preserve aspect ratio (add padding)
  2. Stretch to fit (change aspect ratio)
  3. Crop to fit (maintain aspect ratio)



Enter choice (1, 2, or 3):  3



Counting images...

Summary:
  Directories: 540
  Total images: 162000
  Target size: 112x112
  Method: Crop to fit
  Source: D:\Final_Semester_Project\AI_Attendance_System\ai-ml-model\DataSets\SecondLap\First300



Proceed with resizing 162000 images? (y/n):  y



Processing 162000 images...


Overall Progress: 100%|███████████████████████████████████████████████████████| 162000/162000 [42:59<00:00, 62.80img/s]


RESIZE COMPLETE
Successfully resized: 162000 images
Failed: 0 images
✅ All images processed successfully!

All images resized to 112x112 using: Crop to fit
